In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df_supplier = pd.read_csv('supplier_performance.csv')
df_iqc = pd.read_csv('iqc_inspections.csv')
df_prod = pd.read_csv('ipqc_production_log.csv')
df_fqc = pd.read_csv('fqc_final_test.csv')
df_returns = pd.read_csv('customer_returns.csv')

print("All 5 data files loaded into DataFrames!")

All 5 data files loaded into DataFrames!


## Data Cleaning
The files are loaded. Now I will clean each csv file one by one and I will try to see if there is any issues.

In [2]:
df_supplier.head(10)

,Material_Batch_ID,Supplier_Name,Shipment_Date,Material_Cost
0,B1000,Electro Inc,2025-10-14,627.54
1,B1001,ElectroInc.,2025-10-17,550.74
2,B1002,ElectroInc.,2025-10-13,917.28
3,B1003,Electro Inc,2025-10-27,751.83
4,B1004,QualityParts,2025-10-14,977.47
5,B1005,Component Co,2025-10-30,683.78
6,B1006,Component Co,2025-10-17,969.62
7,B1007,QualityParts,2025-10-27,943.11
8,B1008,ComponentCo,2025-10-13,538.72
9,B1009,QualityParts,2025-10-20,942.55


In [3]:
df_supplier.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Material_Batch_ID  50 non-null     object 
 1   Supplier_Name      50 non-null     object 
 2   Shipment_Date      50 non-null     object 
 3   Material_Cost      50 non-null     float64
dtypes: float64(1), object(3)
memory usage: 1.7+ KB


In [4]:
df_supplier['Supplier_Name'].value_counts()

Supplier_Name
ElectroInc.     10
QualityParts    10
Component Co    10
Electro Inc      8
Logi-Tek         7
ComponentCo      5
Name: count, dtype: int64

In [5]:
supplier_name_change = {
    "Electro Inc" : "ElectroInc.",
    "ComponentCo" : "Component Co"
 }

df_supplier['Supplier_Name'] = df_supplier['Supplier_Name'].replace(supplier_name_change)
df_supplier["Supplier_Name"].value_counts()

Supplier_Name
ElectroInc.     18
Component Co    15
QualityParts    10
Logi-Tek         7
Name: count, dtype: int64

In [6]:
df_supplier["Shipment_Date"] = pd.to_datetime(df_supplier["Shipment_Date"])
df_supplier.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Material_Batch_ID  50 non-null     object        
 1   Supplier_Name      50 non-null     object        
 2   Shipment_Date      50 non-null     datetime64[ns]
 3   Material_Cost      50 non-null     float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 1.7+ KB


## df_supplier: Investigation Results

df_supplier (using .head() and .info()) revealed two key problems:

Shipment_Date is the wrong data type: The .info() output shows it's an object. To do any time-based analysis, I must convert this column to a proper datetime format.

Supplier_Name has hidden, messy data: The .info() shows it's an object with 50 non-null values, which looks clean. But this is a classic. When I run df_supplier['Supplier_Name'].value_counts(), I can see that there are duplicate, misspelled names (like 'Electro Inc.' vs. 'Electro Inc' and 'ComponentCo' vs. 'Component Co').

I fixed this problems. Now I will continue with other csv files.

In [7]:
df_iqc.head(10)

,Batch_ID,IQC_Inspector,Inspection_Timestamp,Resistance_Reading,IQC_Result
0,B1048,IQC-01,23/10/2025 03:46,1.437,Fail
1,B1026,IQC-02,30/10/2025 07:44,1.069,Pass
2,B1004,IQC-03,08/11/2025 19:45,1.488,Pass
3,B1025,IQC-02,28/10/2025 15:29,1.39,PASS
4,B1024,IQC-01,07/11/2025 05:28,1.188,FAIL
5,B1013,IQC-01,20/10/2025 05:47,NaN,PASS
6,B1018,IQC-02,24/10/2025 01:58,1.171,Pass
7,B1038,IQC-02,22/10/2025 03:55,1.073,Fail
8,B1031,IQC-02,07/11/2025 04:47,1.46,Pass
9,B1047,IQC-03,30/10/2025 14:38,1.193,FAIL


In [8]:
df_iqc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Batch_ID              100 non-null    object
 1   IQC_Inspector         100 non-null    object
 2   Inspection_Timestamp  100 non-null    object
 3   Resistance_Reading    99 non-null     object
 4   IQC_Result            100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


In [9]:
df_iqc["IQC_Result"].value_counts()

IQC_Result
PASS     30
Fail     24
FAIL     19
Pass     15
Pass     12
Name: count, dtype: int64

In [10]:
print(df_iqc["Resistance_Reading"].value_counts().tail())

Resistance_Reading
1.301    1
1.146    1
1.199    1
1.191    1
1.41     1
Name: count, dtype: int64


In [11]:
df_iqc.rename(columns={"Batch_ID":"Material_Batch_ID"}, inplace=True)
df_iqc["Inspection_Timestamp"] = pd.to_datetime(df_iqc['Inspection_Timestamp'], format='%d/%m/%Y %H:%M')
df_iqc["Resistance_Reading"] = pd.to_numeric(df_iqc["Resistance_Reading"], errors = "coerce")
df_iqc["IQC_Result"] = df_iqc["IQC_Result"].str.upper().str.strip()

In [12]:
df_iqc.head()

,Material_Batch_ID,IQC_Inspector,Inspection_Timestamp,Resistance_Reading,IQC_Result
0,B1048,IQC-01,2025-10-23 03:46:00,1.437,FAIL
1,B1026,IQC-02,2025-10-30 07:44:00,1.069,PASS
2,B1004,IQC-03,2025-11-08 19:45:00,1.488,PASS
3,B1025,IQC-02,2025-10-28 15:29:00,1.390,PASS
4,B1024,IQC-01,2025-11-07 05:28:00,1.188,FAIL


In [13]:
df_iqc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Material_Batch_ID     100 non-null    object        
 1   IQC_Inspector         100 non-null    object        
 2   Inspection_Timestamp  100 non-null    datetime64[ns]
 3   Resistance_Reading    98 non-null     float64       
 4   IQC_Result            100 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 4.0+ KB


In [14]:
df_prod.head(10)

,Product_Serial_Number,Material_Batch_ID,Production_Line,Build_Date,IPQC_Sensor_Temp,IPQC_Sensor_Pressure
0,SN90000,B1043,Line_C,2025-11-05,78.36,14.63
1,SN90001,B1011,Line_B,2025-11-07,66.60,14.66
2,SN90002,B1011,Line_A,2025-11-08,66.18,14.75
3,SN90003,B1018,Line_C,2025-11-01,76.87,14.56
4,SN90004,B1014,Line_A,2025-11-09,76.58,14.88
5,SN90005,B1017,Line_A,2025-11-04,80.80,14.85
6,SN90006,B1021,Line_B,2025-11-06,66.24,13.94
7,SN90007,B1040,Line_A,2025-11-13,75.71,14.62
8,SN90008,B1049,Line_B,2025-11-11,73.48,14.69
9,SN90009,B1044,Line_C,2025-11-04,69.85,14.60


In [15]:
df_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Product_Serial_Number  500 non-null    object 
 1   Material_Batch_ID      500 non-null    object 
 2   Production_Line        500 non-null    object 
 3   Build_Date             500 non-null    object 
 4   IPQC_Sensor_Temp       500 non-null    float64
 5   IPQC_Sensor_Pressure   500 non-null    float64
dtypes: float64(2), object(4)
memory usage: 23.6+ KB


In [16]:
df_prod.describe()

,IPQC_Sensor_Temp,IPQC_Sensor_Pressure
count,500.000000,500.000000
mean,94.457780,14.688000
std,443.929495,0.199835
min,-100.000000,13.940000
25%,71.367500,14.547500
50%,74.795000,14.670000
75%,78.707500,14.820000
max,9999.000000,15.260000


## df_prod
Clearly there is a lot of problems in this file too. The moment I looked at it I saw the problems with: First Date column is object type. Second IPQC_Sensor_Temp has the value of 9999 and -100 since these degrees are not possible they must be errors. I will fix this issues and take a look again to make sure there is no problems

In [18]:
df_prod["Build_Date"] = pd.to_datetime(df_prod["Build_Date"])
df_prod = df_prod[(df_prod['IPQC_Sensor_Temp'] > 10) & (df_prod['IPQC_Sensor_Temp'] < 100)].copy()

df_prod.info()

<class 'pandas.core.frame.DataFrame'>
Index: 498 entries, 0 to 499
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Product_Serial_Number  498 non-null    object        
 1   Material_Batch_ID      498 non-null    object        
 2   Production_Line        498 non-null    object        
 3   Build_Date             498 non-null    datetime64[ns]
 4   IPQC_Sensor_Temp       498 non-null    float64       
 5   IPQC_Sensor_Pressure   498 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 27.2+ KB


In [19]:
df_prod.describe()

,Build_Date,IPQC_Sensor_Temp,IPQC_Sensor_Pressure
count,498,498.000000,498.000000
mean,2025-11-07 14:21:41.204819200,74.959618,14.687992
min,2025-11-01 00:00:00,60.340000,13.940000
25%,2025-11-04 00:00:00,71.392500,14.550000
50%,2025-11-07 12:00:00,74.795000,14.670000
75%,2025-11-11 00:00:00,78.700000,14.820000
max,2025-11-14 00:00:00,88.460000,15.260000
std,NaN,5.020248,0.200010


In [20]:
df_fqc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Serial_No      480 non-null    object
 1   FQC_Test_Date  480 non-null    object
 2   Final_Result   480 non-null    object
 3   Defect_Code    54 non-null     object
dtypes: object(4)
memory usage: 15.1+ KB


In [22]:
df_fqc.head(10)

,Serial_No,FQC_Test_Date,Final_Result,Defect_Code
0,SN90358,2025-11-20,PASS,NaN
1,SN90475,2025-11-18,PASS,NaN
2,SN90244,2025-11-20,PASS,NaN
3,SN90074,2025-11-22,PASS,NaN
4,SN90246,2025-11-17,PASS,NaN
5,SN90232,2025-11-20,PASS,NaN
6,SN90105,2025-11-19,PASS,NaN
7,SN90082,2025-11-17,PASS,NaN
8,SN90165,2025-11-21,PASS,NaN
9,SN90223,2025-11-23,PASS,NaN


In [23]:
df_fqc.describe()

,Serial_No,FQC_Test_Date,Final_Result,Defect_Code
count,480,480,480,54
unique,480,10,2,5
top,SN90358,2025-11-20,PASS,02B
freq,1,59,426,16


In [25]:
df_fqc["Defect_Code"].value_counts()

Defect_Code
02B                16
Code 02B-Screen    12
Code 01A-Power      9
01A                 9
03C-Case            8
Name: count, dtype: int64

In [26]:
fqc_name_change = {
    "Code 02B-Screen":"02B",
    "Code 01A-Power":"01A",
    "03C-Case":"03C"
}
df_fqc["Defect_Code"] = df_fqc["Defect_Code"].replace(fqc_name_change)

df_fqc.rename(columns={'Serial_No': 'Product_Serial_Number'}, inplace=True)
df_fqc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Product_Serial_Number  480 non-null    object
 1   FQC_Test_Date          480 non-null    object
 2   Final_Result           480 non-null    object
 3   Defect_Code            54 non-null     object
dtypes: object(4)
memory usage: 15.1+ KB


In [27]:
print(df_fqc['Final_Result'].value_counts())

Final_Result
PASS    426
FAIL     54
Name: count, dtype: int64


In [29]:
df_fqc["FQC_Test_Date"] = pd.to_datetime(df_fqc["FQC_Test_Date"])
df_fqc["Final_Result"] = df_fqc["Final_Result"].str.upper().str.strip()
df_fqc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Product_Serial_Number  480 non-null    object        
 1   FQC_Test_Date          480 non-null    datetime64[ns]
 2   Final_Result           480 non-null    object        
 3   Defect_Code            54 non-null     object        
dtypes: datetime64[ns](1), object(3)
memory usage: 15.1+ KB


In [30]:
print(df_fqc['Final_Result'].value_counts())

Final_Result
PASS    426
FAIL     54
Name: count, dtype: int64


In [31]:
df_returns.head(10)

,product_serial,Return_Date,Customer_Complaint
0,SN90094,2025-12-03,Case was cracked
1,SN90154,2025-12-18,Dead pixel
2,SN90111,2025-12-18,Case was cracked
3,SN90315,2025-12-18,Screen dead on arrival
4,SN90496,2025-12-17,Case was cracked
5,SN90493,2025-12-03,Dead pixel
6,SN90099,2025-12-14,Dead pixel
7,SN90250,2025-12-16,Screen dead on arrival
8,SN90172,2025-12-03,Case was cracked
9,SN90247,2025-12-12,Screen dead on arrival


In [32]:
df_returns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   product_serial      20 non-null     object
 1   Return_Date         20 non-null     object
 2   Customer_Complaint  20 non-null     object
dtypes: object(3)
memory usage: 612.0+ bytes


In [34]:
df_returns["Customer_Complaint"].head().unique()

array(['Case was cracked', 'Dead pixel', 'Screen dead on arrival'],
      dtype=object)

In [36]:
df_returns.rename(columns={"product_serial": "Product_Serial_Number"}, inplace=True)
df_returns["Return_Date"] = pd.to_datetime(df_returns["Return_Date"])

df_returns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Product_Serial_Number  20 non-null     object        
 1   Return_Date            20 non-null     datetime64[ns]
 2   Customer_Complaint     20 non-null     object        
dtypes: datetime64[ns](1), object(2)
memory usage: 612.0+ bytes


## Step 3: Aggregate & Integrate the Data
All 5 files are now 100% clean. Now, I must integrate them into one table.

### 3a. The "One-to-Many" Problem

I can't just merge all 5 tables. `df_iqc` has multiple inspections*for a single `Material_Batch_ID`.

The Solution: I will first "roll up" the `df_iqc` table using `.groupby()`. I will create a new, small table (`df_iqc_agg`) that shows the *average* `Resistance_Reading` and the *total number* of `FAIL`s for each unique `Material_Batch_ID`.

This aggregated table will be merged with our other data.

In [37]:
df_iqc_agg = df_iqc.copy()

df_iqc_agg['IQC_Fail_Count'] = df_iqc_agg['IQC_Result'].apply(lambda x: 1 if x == 'FAIL' else 0)


df_iqc_agg = df_iqc_agg.groupby('Material_Batch_ID').agg(
    Mean_Resistance_Reading = ('Resistance_Reading', 'mean'),
    Total_IQC_Fails = ('IQC_Fail_Count', 'sum')
).reset_index()


df_iqc_agg.info()
df_iqc_agg.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Material_Batch_ID        43 non-null     object 
 1   Mean_Resistance_Reading  43 non-null     float64
 2   Total_IQC_Fails          43 non-null     int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ KB


,Material_Batch_ID,Mean_Resistance_Reading,Total_IQC_Fails
0,B1000,1.123,0
1,B1001,1.387,2
2,B1002,1.393,1
3,B1004,1.488,0
4,B1005,1.318,0


### Integrating All 5 Data Sources (The Merges)

I have successfully cleaned all 5 data sources and created my aggregated `df_iqc_agg` table.
Now I will perform the final step: joining all of them into one table.

1.  My "base" table will be `df_prod`, as it contains the `Product_Serial_Number` for each unit.
2.  I will use a series of left merges to add the other data. I'm using `left` joins so I keep every product, even if it wasn't returned or didn't have FQC data.

This will create our final "Single Source of Truth."

In [38]:
df_master = df_prod.copy()

df_master = pd.merge(df_master, df_fqc, on="Product_Serial_Number", how="left")

df_master = pd.merge(df_master, df_returns, on="Product_Serial_Number", how="left")

df_master = pd.merge(df_master, df_supplier, on="Material_Batch_ID", how="left")

df_master = pd.merge(df_master, df_iqc_agg, on="Material_Batch_ID", how="left")

print("Let's look at results")
df_master.info()

df_master.head()

Let's look at results
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Product_Serial_Number    498 non-null    object        
 1   Material_Batch_ID        498 non-null    object        
 2   Production_Line          498 non-null    object        
 3   Build_Date               498 non-null    datetime64[ns]
 4   IPQC_Sensor_Temp         498 non-null    float64       
 5   IPQC_Sensor_Pressure     498 non-null    float64       
 6   FQC_Test_Date            478 non-null    datetime64[ns]
 7   Final_Result             478 non-null    object        
 8   Defect_Code              54 non-null     object        
 9   Return_Date              20 non-null     datetime64[ns]
 10  Customer_Complaint       20 non-null     object        
 11  Supplier_Name            498 non-null    object        
 12  Shipment_Date 

,Product_Serial_Number,Material_Batch_ID,Production_Line,Build_Date,IPQC_Sensor_Temp,IPQC_Sensor_Pressure,FQC_Test_Date,Final_Result,Defect_Code,Return_Date,Customer_Complaint,Supplier_Name,Shipment_Date,Material_Cost,Mean_Resistance_Reading,Total_IQC_Fails
0,SN90000,B1043,Line_C,2025-11-05,78.36,14.63,2025-11-18,PASS,NaN,NaT,NaN,ElectroInc.,2025-10-23,982.31,NaN,NaN
1,SN90001,B1011,Line_B,2025-11-07,66.60,14.66,2025-11-21,PASS,NaN,NaT,NaN,Component Co,2025-10-01,977.87,1.150,2.0
2,SN90002,B1011,Line_A,2025-11-08,66.18,14.75,2025-11-15,PASS,NaN,NaT,NaN,Component Co,2025-10-01,977.87,1.150,2.0
3,SN90003,B1018,Line_C,2025-11-01,76.87,14.56,2025-11-15,PASS,NaN,NaT,NaN,Logi-Tek,2025-10-27,944.48,1.175,1.0
4,SN90004,B1014,Line_A,2025-11-09,76.58,14.88,2025-11-16,PASS,NaN,NaT,NaN,Logi-Tek,2025-10-20,744.25,NaN,NaN


My Merge is a success! I now have one `df_master` table.

Looking at the `.head()` output, the `NaN` values are not errors—they are the:

* **`NaN` in `Return_Date` & `Customer_Complaint`:** This is good. It means these products were `PASS` and were not returned by a customer.
* **`NaN` in `Mean_Resistance_Reading` (Rows 0 & 4):** This is our first insight. It means `Material_Batch_ID` `B1043` and `B1014` have no IQC inspection data on file. This is a data gap in the company.
* **`2.0` in `Total_IQC_Fails` (Rows 1 & 2):** This is our second insight! It means `Material_Batch_ID` `B1011` (used for products SN90001 and SN90002) failed two IQC inspections before it was used. This is a potential root cause for a defect!

I have successfully integrated all 5 data sources.

## Export the Clean Data

I have finished the entire "Extract, Transform, Integrate" process.

My `df_master` table is the final, clean "Single Source of Truth." I will now export it to a new, clean CSV file. This is the one file I will use to build my Power BI (or Tableau) dashboard.

In [39]:
df_master.to_csv('master_quality_dataset.csv', index=False)

Thanks for reading all the way here. 